In [1]:
!pip install fuzzywuzzy[speedup]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.2 MB/s eta 0:00:00


In [2]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 10.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [3]:
pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.7 MB/s eta 0:00:00


In [4]:
import json
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import numpy as np
from google.colab import drive
from urllib.robotparser import RobotFileParser
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
from textblob import TextBlob
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
from transformers import pipeline
from sklearn.preprocessing import LabelEncoder
from nltk.sentiment import SentimentIntensityAnalyzer
from gensim.models import Word2Vec
from tqdm import tqdm
from fuzzywuzzy import fuzz
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.decomposition import LatentDirichletAllocation
from transformers import BertTokenizer, BertModel

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
path = "/content/drive/My Drive/Fuld/Raw Data v4.json"

In [7]:
with open(path, "r", encoding = "utf-8") as file:
  data = json.load(file)


In [8]:
print(data[:3])

[{'title': "South African fashion retailer TFG buys UK's White Stuff", 'link': 'https://www.reuters.com/markets/deals/south-african-fashion-retailer-tfg-buys-uks-white-stuff-2024-10-25/', 'photo_url': 'https://www.reuters.com/resizer/v2/2KOP6CTZDZKEVJW2GD4EBEWLFA.jpg?auth=1cd9323c857290811636ec34b37c1f376cb915290f11ca5ebda3899213ac910b&width=4918&quality=80', 'thumbnail_url': 'https://news.google.com/api/attachments/CC8iK0NnNVBhRXRJUW5odFIwcHJOa1F6VFJDOEF4aXpCU2dLTWdhRlVwQW1MUWM=-w200-h200-p-df-rw', 'published_datetime_utc': '2024-10-25T07:00:00.000Z', 'authors': "['Louise Heavens', 'Emelia Sithole-Matarise', 'Nqobile Dludla']", 'source_url': 'https://www.reuters.com', 'source_name': 'Reuters', 'source_logo_url': 'https://lh3.googleusercontent.com/OzNZmYwP-Zl3sTWa8dzzyws9hou0ZihOGJ_-qanon-1iB5KwyqcHOwPxYZj54cGMZwD5qXui', 'source_favicon_url': 'https://encrypted-tbn2.gstatic.com/faviconV2?url=https://www.reuters.com&client=NEWS_360&size=96&type=FAVICON&fallback_opts=TYPE,SIZE,URL', 'sou

In [9]:
DF = pd.DataFrame(data)

In [10]:
DF.head()

,title,link,photo_url,thumbnail_url,published_datetime_utc,authors,source_url,source_name,source_logo_url,source_favicon_url,source_publication_id,related_topics,Full_Article
0,South African fashion retailer TFG buys UK's W...,https://www.reuters.com/markets/deals/south-af...,https://www.reuters.com/resizer/v2/2KOP6CTZDZK...,https://news.google.com/api/attachments/CC8iK0...,2024-10-25T07:00:00.000Z,"['Louise Heavens', 'Emelia Sithole-Matarise', ...",https://www.reuters.com,Reuters,https://lh3.googleusercontent.com/OzNZmYwP-Zl3...,https://encrypted-tbn2.gstatic.com/faviconV2?u...,CAAqBggKMLegDDCwJg,[{'topic_id': 'CAAqKAgKIiJDQkFTRXdvTkwyY3ZNVEZ...,None
1,Hobbs-owner acquires White Stuff in bid to boo...,https://www.just-style.com/news/hobbs-owner-aq...,https://www.just-style.com/wp-content/uploads/...,https://news.google.com/api/attachments/CC8iK0...,2024-10-28T07:00:00.000Z,['Shemona Safaya'],https://www.just-style.com,just-style.com,None,https://encrypted-tbn3.gstatic.com/faviconV2?u...,CAAqKQgKIiNDQklTRkFnTWFoQUtEbXAxYzNRdGMzUjViR1...,[{'topic_id': 'CAAqKAgKIiJDQkFTRXdvTkwyY3ZNVEY...,"TFG London, the parent company behind Hobbs, P..."
2,White Stuff dress with pockets that 'hides lum...,https://www.nottinghampost.com/whats-on/shoppi...,https://i2-prod.nottinghampost.com/whats-on/sh...,https://news.google.com/api/attachments/CC8iK0...,2025-02-03T11:48:23.000Z,['Lucy Addicott'],https://www.nottinghampost.com,Nottinghamshire Live,https://lh3.googleusercontent.com/msWuiz7du-FG...,https://encrypted-tbn3.gstatic.com/faviconV2?u...,CAAqBwgKMIfCoQkwid2WAg,[{'topic_id': 'CAAqKAgKIiJDQkFTRXdvTkwyY3ZNVEZ...,"As we head into February, fashion brand White ..."
3,South Africa’s TFG acquires White Stuff,https://www.retail-insight-network.com/news/tf...,https://www.retail-insight-network.com/wp-cont...,https://news.google.com/api/attachments/CC8iL0...,2024-10-28T07:00:00.000Z,[],https://www.retail-insight-network.com,Retail Insight Network,None,https://encrypted-tbn1.gstatic.com/faviconV2?u...,CAAqBwgKMKz4nQswvYK2Aw,[],"South African retail group TFG, through its Lo..."
4,Fiftysomethings in pink bobble jumpers: how Wh...,https://www.theguardian.com/business/article/2...,https://i.guim.co.uk/img/media/9f854e09421b65d...,https://news.google.com/api/attachments/CC8iK0...,2024-06-04T07:00:00.000Z,['Sarah Butler'],https://www.theguardian.com,The Guardian,https://lh3.googleusercontent.com/SU8FywVNtcrG...,https://encrypted-tbn0.gstatic.com/faviconV2?u...,CAAqBggKMJeqezDfswk,[{'topic_id': 'CAAqKAgKIiJDQkFTRXdvTkwyY3ZNVEZ...,"‘I’m just getting going,” says Jo Jenkins, the..."


In [11]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1418 entries, 0 to 1417
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   title                   1418 non-null   object
 1   link                    1418 non-null   object
 2   photo_url               1369 non-null   object
 3   thumbnail_url           1369 non-null   object
 4   published_datetime_utc  1418 non-null   object
 5   authors                 1418 non-null   object
 6   source_url              1418 non-null   object
 7   source_name             1418 non-null   object
 8   source_logo_url         640 non-null    object
 9   source_favicon_url      1418 non-null   object
 10  source_publication_id   1418 non-null   object
 11  related_topics          1418 non-null   object
 12  Full_Article            1157 non-null   object
dtypes: object(13)
memory usage: 144.1+ KB


In [12]:
# Convert published date to datetime format
DF["published_datetime_utc"] = pd.to_datetime(DF["published_datetime_utc"], errors="coerce")

In [13]:
# Filter articles from 2023 and 2024
DF = DF[(DF["published_datetime_utc"].dt.year >= 2023) & (DF["published_datetime_utc"].dt.year <= 2024)]

In [14]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 854 entries, 0 to 1415
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   title                   854 non-null    object             
 1   link                    854 non-null    object             
 2   photo_url               833 non-null    object             
 3   thumbnail_url           833 non-null    object             
 4   published_datetime_utc  854 non-null    datetime64[ns, UTC]
 5   authors                 854 non-null    object             
 6   source_url              854 non-null    object             
 7   source_name             854 non-null    object             
 8   source_logo_url         435 non-null    object             
 9   source_favicon_url      854 non-null    object             
 10  source_publication_id   854 non-null    object             
 11  related_topics          854 non-null    object   

#It seems that there are missing records in column Full Article. Hence, we need to do web scrapping for those missing articles.

In [15]:
missing_articles = DF[DF["Full_Article"].isnull()]

In [16]:
print(f"Total missing articles: {len(missing_articles)}")

Total missing articles: 163


In [17]:
# Function to scrape an article
def scrape_article(url):
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers, timeout=10)  # Ensuring timeout
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract article text (modify this logic based on the site's structure)
        paragraphs = soup.find_all("p")
        article_text = "\n".join([p.get_text() for p in paragraphs])

        return article_text.strip() if article_text.strip() else None

    except requests.exceptions.RequestException as e:
        print(f"Error scraping {url}: {e}")
        return None  # Return None on failure

In [18]:
# Function to process one article
def process_article(idx, url):
    print(f"Scraping: {url}")
    article_text = scrape_article(url)
    return idx, article_text

In [19]:
# Multi-threading with a timeout to prevent infinite loops
with ThreadPoolExecutor(max_workers=5) as executor:
    future_to_url = {executor.submit(process_article, row[0], row[1]["link"]): row[0] for row in missing_articles.iterrows()}

    results = []
    for future in as_completed(future_to_url, timeout=300):  # Global timeout of 300s
        try:
            idx, article_text = future.result(timeout=15)  # Each request has max 15s
            results.append((idx, article_text))
        except Exception as e:
            print(f"Thread failed: {e}")

# Store results in DataFrame
for idx, article_text in results:
    DF.at[idx, "Full_Article"] = article_text

# Save progress
DF.to_csv("Updated_Articles.csv", index=False, encoding="utf-8")
print(f"Scraping complete. Data saved. {len(results)} successful, {len(missing_articles) - len(results)} failed.")

Scraping: https://www.reuters.com/markets/deals/south-african-fashion-retailer-tfg-buys-uks-white-stuff-2024-10-25/
Scraping: https://www.foodepedia.co.uk/restaurant-news/the-white-stuff-german-gymnasium-serves-white-asparagus/
Scraping: https://www.stuff.co.nz/world-news/350452570/what-are-these-mysterious-blobs-washing-canadian-beaches
Scraping: https://www.stuff.co.nz/sport/360482689/all-whites-secure-biggest-win-two-decades-world-cup-qualifier-against-vanuatu
Scraping: https://www.stuff.co.nz/home-property/360475544/leading-landscape-designer-bucked-sexist-stereotype
Scraping: https://www.stuff.co.nz/nz-news/350262054/new-details-revealed-after-young-mother-killed-boating-incident
Scraping: https://www.stuff.co.nz/home-property/350177734/white-paint-through-decades-remember-half-tea-one-we-love-now
Scraping: https://www.stuff.co.nz/nz-news/350293195/misery-vines-tale-exploitation
Error scraping https://www.reuters.com/markets/deals/south-african-fashion-retailer-tfg-buys-uks-white-

In [20]:
# Reload the updated DataFrame
df_updated = pd.read_csv("Updated_Articles.csv", encoding="utf-8")

In [21]:
#df_updated["published_datetime_utc"] = pd.to_datetime(df_updated["published_datetime_utc"], errors="coerce")

In [22]:
# Extract the year
#df_updated["year"] = df_updated["published_datetime_utc"].dt.year

# Find unique years present in the dataset
#unique_years = df_updated["year"].dropna().unique()

#print("Unique years in the dataset:", unique_years)

In [23]:
#files.download("Updated_Articles.csv")

In [24]:
#df_updated.info()

In [25]:
#df = pd.read_csv("Updated_Articles.csv", delimiter=",", encoding="utf-8", on_bad_lines="skip")
#print(df.shape)  # Check number of rows and columns

In [26]:
# Find missing articles again
missing_articles_after = df_updated[df_updated["Full_Article"].isnull()]

In [27]:
# Print the number of missing articles
print(f"Total missing articles after scraping: {len(missing_articles_after)}")

Total missing articles after scraping: 105


In [28]:
df_updated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 854 entries, 0 to 853
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   title                   854 non-null    object
 1   link                    854 non-null    object
 2   photo_url               833 non-null    object
 3   thumbnail_url           833 non-null    object
 4   published_datetime_utc  854 non-null    object
 5   authors                 854 non-null    object
 6   source_url              854 non-null    object
 7   source_name             854 non-null    object
 8   source_logo_url         435 non-null    object
 9   source_favicon_url      854 non-null    object
 10  source_publication_id   854 non-null    object
 11  related_topics          854 non-null    object
 12  Full_Article            749 non-null    object
dtypes: object(13)
memory usage: 86.9+ KB


Before web scrapping, We had 163 missing articles and now we have 74 missing articles. 74 missing articles were not captured due to either article is no longer on website or Website Blocks Scraping. Therefore, we will proceed with available date

In [29]:
# Remove rows where 'Full_Article' is null
df_updated.dropna(subset=["Full_Article"], inplace = True)

# Reset index after dropping rows
df_updated.reset_index(drop=True, inplace=True)

# Check updated dataset size
print(f"Updated dataset size: {len(df_updated)} (Removed {854 - len(df_updated)} missing articles)")

# Save the cleaned dataset
df_updated.to_csv("Updated_Articles.csv", index=False, encoding="utf-8")
print("Updated dataset saved as 'Updated_Articles.csv'.")

Updated dataset size: 749 (Removed 105 missing articles)
Updated dataset saved as 'Updated_Articles.csv'.


#Lets filter out the articles based on given suppliers in assignment

In [30]:
# Load spaCy's Named Entity Recognition (NER) model
nlp = spacy.load("en_core_web_sm")

In [31]:
df1 = pd.read_csv("Updated_Articles.csv")

In [32]:
# List of supplier names (for exact and fuzzy matching)
suppliers = [
    "Welspun Living Limited",
    "Teejay Lanka PLC",
    "Arvind Limited",
    "Caleres, Inc.",
    "Interloop Limited",
    "Kitex Garments Limited",
    "ThredUp Inc.",
    "G-III Apparel Group, Ltd.",
    "Mint Velvet",
    "White Stuff Limited"
]

In [33]:
def extract_suppliers(text):
  if pd.isna(text):  # Handle NaN values
        return []

  found_suppliers = set()

  # Named Entity Recognition (NER) for organizations
  doc = nlp(text)
  for ent in doc.ents:
          if ent.label_ == "ORG":  # Check if the entity is an organization
                      for supplier in suppliers:
                                        if supplier.lower() in ent.text.lower():  # Exact match
                                          found_suppliers.add(supplier)
                                        elif fuzz.partial_ratio(supplier.lower(), ent.text.lower()) > 85:  # Fuzzy matching
                                          found_suppliers.add(supplier)

  return list(found_suppliers)

In [34]:
# Apply function to extract suppliers from the 'content' column
df1["suppliers"] = df1["Full_Article"].apply(extract_suppliers)

In [35]:
# Filter only rows where suppliers were identified
filtered_df = df1[df1["suppliers"].apply(lambda x: len(x) > 0)]

In [36]:
# Save filtered data
filtered_df.to_csv("Filtered_Articles.csv", index=False)



In [37]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 544 entries, 0 to 747
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   title                   544 non-null    object
 1   link                    544 non-null    object
 2   photo_url               534 non-null    object
 3   thumbnail_url           534 non-null    object
 4   published_datetime_utc  544 non-null    object
 5   authors                 544 non-null    object
 6   source_url              544 non-null    object
 7   source_name             544 non-null    object
 8   source_logo_url         244 non-null    object
 9   source_favicon_url      544 non-null    object
 10  source_publication_id   544 non-null    object
 11  related_topics          544 non-null    object
 12  Full_Article            544 non-null    object
 13  suppliers               544 non-null    object
dtypes: object(14)
memory usage: 63.8+ KB


In [38]:
df = filtered_df[['title','Full_Article']]

In [39]:
df.head()

,title,Full_Article
0,Hobbs-owner acquires White Stuff in bid to boo...,"TFG London, the parent company behind Hobbs, P..."
1,South Africa’s TFG acquires White Stuff,"South African retail group TFG, through its Lo..."
2,Fiftysomethings in pink bobble jumpers: how Wh...,"‘I’m just getting going,” says Jo Jenkins, the..."
3,"TFG acquires fashion and lifestyle retailer, W...","TFG acquires fashion and lifestyle retailer, W..."
4,Whistles owner The Foschini Group buys White S...,Whistles owner The Foschini Group buys White S...


In [40]:
df.tail()

,title,Full_Article
743,Caleres Reports Second Quarter Results and Upd...,ST. LOUIS--(BUSINESS WIRE)--Caleres (NYSE: CAL...
744,Caleres Names Liz Dunn SVP of Corporate Develo...,ST. LOUIS--(BUSINESS WIRE)--Caleres (NYSE: CAL...
745,"Is Caleres, Inc. (NYSE:CAL) Potentially Underv...","In This Article:\nWhile Caleres, Inc. (NYSE:CA..."
746,Caleres Reports Second Quarter Results and Upd...,ST. LOUIS--(BUSINESS WIRE)--Caleres (NYSE: CAL...
747,Does Caleres (NYSE:CAL) Have A Healthy Balance...,"Some say volatility, rather than debt, is the ..."


In [41]:
nltk.download("averaged_perceptron_tagger_eng")  # Specify 'eng' for English
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")
nltk.download('punkt_tab')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [42]:
# Initialize NLP tools
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

In [43]:
# Function to convert POS tags to WordNet format
def get_wordnet_pos(word):
    tag = pos_tag([word])[0][1][0].upper()  # Get first letter of POS tag
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)  # Default to noun if not found

In [44]:
def preprocess_text(text):
    if pd.isnull(text):  # Handle missing values
        return ""

    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # Convert to lowercase
    text = text.lower()

    # Remove special characters, numbers, and extra spaces
    text = re.sub(r"[^a-z\s]", "", text)

    # Tokenization
    words = word_tokenize(text)

    # Remove stopwords
    words = [word for word in words if word not in stop_words]

    # Lemmatization with POS tagging
    words = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in words]

    return " ".join(words)

# Apply preprocessing to the Full_Article column
df["Cleaned_Article"] = df["Full_Article"].apply(preprocess_text)


<ipython-input-44-aa461f298f04>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Cleaned_Article"] = df["Full_Article"].apply(preprocess_text)


In [45]:
import spacy
# Load Spacy NLP model with necessary components
#nlp = spacy.load("en_core_web_sm", disable=["parser"])  # Keep 'tagger' for lemmatization

In [46]:
def extract_named_entities_batch(texts):
    """Extract Named Entities in batches for efficiency."""
    entities_list = []
    for doc in nlp.pipe(texts, batch_size=20, disable=["parser"]):  # Batch processing for speed
        entities = [(ent.text, ent.label_) for ent in doc.ents]
        entities_list.append(entities)
    return entities_list



In [47]:
# Load the dataframe
#df = pd.read_csv("Updated_Articles.csv")

In [48]:
# Apply NER on the cleaned article column
df.loc[:, "Named_Entities"] = df["Cleaned_Article"].astype(str).apply(lambda text: [(ent.text, ent.label_) for ent in nlp(text).ents])

<ipython-input-48-6cd14053ff7c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "Named_Entities"] = df["Cleaned_Article"].astype(str).apply(lambda text: [(ent.text, ent.label_) for ent in nlp(text).ents])


In [49]:
# Apply Sentiment Analysis on the cleaned article column, handling missing values
df.loc[:, "Sentiment_Score"] = df["Cleaned_Article"].astype(str).apply(lambda text: TextBlob(text).sentiment.polarity if not pd.isnull(text) else np.nan)

<ipython-input-49-517820218677>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "Sentiment_Score"] = df["Cleaned_Article"].astype(str).apply(lambda text: TextBlob(text).sentiment.polarity if not pd.isnull(text) else np.nan)


In [50]:
from collections import Counter

# Function to get word frequencies
def get_word_frequencies(text):
    # Check if text is a valid string before splitting
    if isinstance(text, str):
        words = text.split()
        word_counts = Counter(words)
        return dict(word_counts)
    else:
        return {}  # Return an empty dictionary for non-string values

# Apply the word frequency function to the 'Cleaned_Article' column
df.loc[:, "Word_Frequency"] = df["Cleaned_Article"].apply(get_word_frequencies)

<ipython-input-50-df05ccdd9d34>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "Word_Frequency"] = df["Cleaned_Article"].apply(get_word_frequencies)


In [51]:
df.to_csv("/content/drive/My Drive/Preprocessed2_Articles.csv", index=False, encoding="utf-8")

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 544 entries, 0 to 747
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            544 non-null    object 
 1   Full_Article     544 non-null    object 
 2   Cleaned_Article  544 non-null    object 
 3   Named_Entities   544 non-null    object 
 4   Sentiment_Score  544 non-null    float64
 5   Word_Frequency   544 non-null    object 
dtypes: float64(1), object(5)
memory usage: 29.8+ KB


In [53]:
# 4. **Topic Modeling (TF-IDF)**
vectorizer = TfidfVectorizer(max_features=50)  # Adjust number of features as needed
tfidf_matrix = vectorizer.fit_transform(df["Cleaned_Article"])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Merge TF-IDF features into the main dataset
df = pd.concat([df, tfidf_df], axis=1)

In [54]:
df.to_csv("/content/drive/My Drive/Preprocessed1_Articles.csv", index=False, encoding="utf-8")

In [55]:
# Download required NLTK data
nltk.download("vader_lexicon")
nltk.download("punkt")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [56]:
# Create an empty 'Risk_Category' column
#df['Risk_Category'] = np.nan

In [57]:
# Select a random sample of 30 articles for manual labeling
manual_label_subset = df.sample(30, random_state=42)

In [58]:
manual_label_subset = manual_label_subset.loc[:, ["title", "Cleaned_Article"]]
manual_label_subset["Risk_Category"] = ""

In [59]:
# Save as Excel file (fixing multi-line text issue)
manual_label_subset.to_excel("manual_labeling_template.xlsx", index=False, engine="openpyxl")

In [60]:
# Drop empty rows if necessary
manual_label_subset = manual_label_subset.dropna()

In [61]:
from google.colab import files

In [62]:
#files.download("manual_labeling_template.xlsx")

In [63]:
from google.colab import files

In [64]:
uploaded = files.upload()

Saving manual_labeling_template (1).xlsx to manual_labeling_template (1).xlsx


In [65]:
manual_labels = pd.read_excel("manual_labeling_template (1).xlsx")

In [66]:
manual_labels.head()

,title,Cleaned_Article,Risk_Category
0,Caleres Reports Second Quarter Results and Upd...,st louisbusiness wirecaleres nyse cal marketle...,Financial and Operational Risks
1,Interloop’s Pakistan Plant Earns LEED Platinum...,interloops new apparel plant pakistan receive ...,Agricultural and Environmental Risks
2,Caleres Inc (CAL) Q2 2024 Earnings: EPS of $0....,september caleres inc cal financial release k ...,Geopolitical and Regulatory Risks
3,"Welspun Living Buyback 2024 Record Date, Price...",welspun living crore approve board director me...,Financial and Operational Risks
4,VIS upgrades Interloop’s ratings,july mln vi credit rating company limited vi u...,Financial and Operational Risks


In [67]:
df = df.merge(manual_labels[["title", "Risk_Category"]], on="title", how="left")
# Fill missing risk categories as 'Unlabeled'
df["Risk_Category"] = df["Risk_Category"].fillna("Unlabeled")

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671 entries, 0 to 670
Data columns (total 57 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            544 non-null    object 
 1   Full_Article     544 non-null    object 
 2   Cleaned_Article  544 non-null    object 
 3   Named_Entities   544 non-null    object 
 4   Sentiment_Score  544 non-null    float64
 5   Word_Frequency   544 non-null    object 
 6   ai               544 non-null    float64
 7   also             544 non-null    float64
 8   august           544 non-null    float64
 9   bank             544 non-null    float64
 10  brand            544 non-null    float64
 11  business         544 non-null    float64
 12  company          544 non-null    float64
 13  crore            544 non-null    float64
 14  earnings         544 non-null    float64
 15  financial        544 non-null    float64
 16  global           544 non-null    float64
 17  growth          

In [69]:
df["Risk_Category"].value_counts()

,count
Risk_Category,
Unlabeled,610
Financial and Operational Risks,28
Market and Competitive Risks,25
Supply Chain and Logistics Risks,3
Agricultural and Environmental Risks,3
Geopolitical and Regulatory Risks,2


In [70]:
df.to_csv("/content/drive/My Drive/Updated_Articles.csv", index=False, encoding="utf-8")

See Kaggle Notebook for remaining code. This project was run on Kaggle due to GPU issue